In [ ]:
import sqlite3
import pandas as pd

# Connect to the Chinook SQLite database
conn = sqlite3.connect("Chinook.sqlite")  # Ensure the filename matches exactly

# Define the SQL query
query = """
SELECT 
    Customer.LastName, 
    Customer.FirstName, 
    Track.Name AS TrackName, 
    Album.Title AS AlbumTitle
FROM Customer
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
JOIN Track ON InvoiceLine.TrackId = Track.TrackId
JOIN Album ON Track.AlbumId = Album.AlbumId
ORDER BY Customer.LastName, Customer.FirstName;
"""

# Load the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Display the first few rows
print(df.head())  # To check the output


In [2]:
import sqlite3
import pandas as pd

# Connect to the Chinook SQLite database
conn = sqlite3.connect("Chinook.sqlite")  # Ensure the filename is correct

# Define the SQL query
query = """
SELECT 
    Customer.LastName, 
    Customer.FirstName, 
    Track.Name AS TrackName, 
    Album.Title AS AlbumTitle
FROM Customer
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
JOIN Track ON InvoiceLine.TrackId = Track.TrackId
JOIN Album ON Track.AlbumId = Album.AlbumId
ORDER BY Customer.LastName, Customer.FirstName;
"""

# Load the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Display the first few rows
print("📌 First few rows of the dataset:")
print(df.head())

# Check data completeness
print("\n📌 Dataset Shape (Rows, Columns):", df.shape)

# Count unique customers
unique_customers = df[['LastName', 'FirstName']].nunique()
print("\n📌 Unique Customers Count:")
print(unique_customers)

# Group by customer and count purchases
customer_purchases = df.groupby(['LastName', 'FirstName']).size().reset_index(name='Total Purchases')

# Display top customers by purchase count
print("\n📌 Top 10 Customers by Number of Purchases:")
print(customer_purchases.sort_values(by='Total Purchases', ascending=False).head(10))

# Export data to CSV
df.to_csv("customer_purchases.csv", index=False)
print("\n✅ Data exported to 'customer_purchases.csv'")


📌 First few rows of the dataset:
  LastName FirstName                                          TrackName  \
0  Almeida   Roberto                            Right Next Door to Hell   
1  Almeida   Roberto                                     In The Evening   
2  Almeida   Roberto                                   Fool In The Rain   
3  Almeida   Roberto  Saudade Dos Aviões Da Panair (Conversando No Bar)   
4  Almeida   Roberto                             Caso Você Queira Saber   

                AlbumTitle  
0      Use Your Illusion I  
1  In Through The Out Door  
2  In Through The Out Door  
3                    Minas  
4                    Minas  

📌 Dataset Shape (Rows, Columns): (2240, 4)

📌 Unique Customers Count:
LastName     59
FirstName    57
dtype: int64

📌 Top 10 Customers by Number of Purchases:
    LastName  FirstName  Total Purchases
0    Almeida    Roberto               38
43     Rocha  Alexandre               38
31    Miller        Dan               38
32  Mitchell      